In [7]:
import os
import requests

In [12]:
from dotenv import load_dotenv

# Carga las variables de entorno desde el archivo .env
load_dotenv()

# Accede a la contraseña desde las variables de entorno
password = os.getenv('COPERNICUS_PASSWORD')
username = os.getenv('COPERNICUS_USERNAME')

In [105]:
import urllib.request
from urllib.error import HTTPError

def download_nc_file_with_auth(url, output_file, username, password):
    # Create a password manager with the given username and password
    password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
    password_mgr.add_password(None, url, username, password)
    handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
    opener = urllib.request.build_opener(handler)
    
    try:
        # Open the URL with the configured opener
        response = opener.open(url)
        
        with open(output_file, 'wb') as f:
            f.write(response.read())
        
        print(f"Downloaded {url} to {output_file}")
    except HTTPError as e:
        print(f"Failed to download {url}. Status code: {e.code}, Reason: {e.reason}")

In [104]:
link_1 = f"https://land.copernicus.vgt.vito.be/manifest/ndvi_v3_1km/manifest_cgls_ndvi_v3_1km_latest.txt" #2020 - 1999
link_2 = f"https://land.copernicus.vgt.vito.be/manifest/ssm_v1_1km/manifest_cgls_ssm_v1_1km_latest.txt" #SSM 2014
link_3 = f"https://land.copernicus.vgt.vito.be/manifest/fapar_v2_1km/manifest_cgls_fapar_v2_1km_latest.txt" #FAPAR 1999 - 202006

link_4 = f"https://land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Indicators/NDVI_1km_V2_LTS/1999/12/21/NDVI-LTS_1999-2017-1221_GLOBE_VGT-PROBAV_V2.2.1/c_gls_NDVI-LTS_1999-2017-1221_GLOBE_VGT-PROBAV_V2.2.1.nc"

links = [link_1, link_2, link_3]
folders = ["nvdi", "ssm", "fapar"]

link_dict = {link:folder for link, folder in zip(links, folders)}

main_data_folder = f"../data"

In [106]:


for link, folder in link_dict.items():

    data_folder = f"{main_data_folder}/{folder}"
    if os.path.exists(data_folder) is False:
        os.mkdir(data_folder)

    response = requests.get(link_1)

    response.raise_for_status()

    nc_files = response.text.split("\n")

    for nc_file in nc_files:

        filename = nc_file.split("/")[-1]

        if ".nc" in filename:
            output_filename = f'{data_folder}/{filename}.nc'
            download_nc_file_with_auth(nc_file, output_filename, username, password)

        else:
            continue


        break

    break

In [29]:
import netCDF4 as nc

ds = nc.Dataset("test.nc")

In [56]:
ds.variables["lat"][0]

masked_array(data=80.,
             mask=False,
       fill_value=1e+20)

In [59]:
ds.variables['NDVI'][1000]

masked_array(data=[--, --, --, ..., --, --, --],
             mask=[ True,  True,  True, ...,  True,  True,  True],
       fill_value=255,
            dtype=float32)